<img width="800px" src="../fidle/img/header.svg"></img>

# <!-- TITLE --> [LVAE2] - VAE, using a custom model class  (MNIST dataset)
<!-- DESC --> Construction and training of a VAE, using model subclass, with a latent space of small dimension, using PyTorch Lightninh

<!-- AUTHOR : Achille Mbogol Touye (EFILIA-MIAI/SIMaP) -->

## Objectives :
 - Understanding and implementing a **variational autoencoder** neurals network (VAE)
 - Understanding a still more **advanced programming model**, using a **custom model**

The calculation needs being important, it is preferable to use a very simple dataset such as MNIST to start with.  
...MNIST with a small scale if you haven't a GPU ;-)

## What we're going to do :

 - Defining a VAE model
 - Build the model
 - Train it
 - Have a look on the train process


## Step 1 - Init python stuff

In [ ]:
import os,sys
import torch
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import scipy.stats

import torch.nn as nn
import lightning.pytorch as pl

from torch.utils.data    import TensorDataset, DataLoader
from modules.callbacks   import ImagesCallback,BestModelCallback
from modules.progressbar import CustomTrainProgressBar
from modules.layers      import SamplingLayer
from modules.datagen     import MNIST
from modules.models      import VAE, Encoder, Decoder
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger

import fidle

# Init Fidle environment
run_id, run_dir, datasets_dir = fidle.init('LVAE2')

VAE.about()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Step 2 - Parameters
`scale` : with scale=1, we need 1'30s on a GPU V100 ...and >20' on a CPU !  
`latent_dim` : 2 dimensions is small, but usefull to draw !  
`fit_verbosity`: Verbosity of training progress bar: 0=silent, 1=progress bar, 2=One line  

`loss_weights` : Our **loss function** is the weighted sum of two loss:
 - `r_loss` which measures the loss during reconstruction.  
 - `kl_loss` which measures the dispersion.  

The weights are defined by: `loss_weights=[k1,k2]` where : `total_loss = k1*r_loss + k2*kl_loss`  
In practice, a value of \[1,.01\] gives good results here.


In [ ]:
latent_dim    = 6
loss_weights  = [1,.001]       # [1, .001] give good results

scale         = .2
seed          = 123

batch_size    = 64
epochs        = 5
fit_verbosity = 1

Override parameters (batch mode) - Just forget this cell

In [ ]:
fidle.override('latent_dim', 'loss_weights', 'scale', 'seed', 'batch_size', 'epochs', 'fit_verbosity')

## Step 3 - Prepare data
`MNIST.get_data()` return : `x_train,y_train, x_test,y_test`,  \
but we only need x_train for our training.

In [ ]:
x_data, y_data, _,_ = MNIST.get_data(seed=seed, scale=scale, train_prop=1 )

fidle.scrawler.images(x_data[:20], None, indices='all', columns=10, x_size=1,y_size=1,y_padding=0, save_as='01-original')

 ## 3.1 -  For Training model use Dataloader
The Dataset retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in minibatches, reshuffle the data at every epoch to reduce model overfitting. DataLoader is an iterable that abstracts this complexity for us in an easy API

In [ ]:
train_dataset = TensorDataset(x_data,y_data)

# train bacth data
train_loader= DataLoader(
  dataset=train_dataset, 
  shuffle=False, 
  batch_size=batch_size, 
  num_workers=2 
)

## Step 4 - Build model
In this example, we will use a **custom model**.
For this, we will use :
 - `SamplingLayer`, which generates a vector z from the parameters z_mean and z_logvar - See : [SamplingLayer.py](./modules/layers/SamplingLayer.py)
 - `VAE`, a custom model- See : [VAE.py](./modules/models/VAE.py)

#### VAE
`VAE` is a custom model with a specific train_step - See : [VAE.py](./modules/models/VAE.py)

In [ ]:
vae=VAE(Encoder(latent_dim = latent_dim),
        Decoder(latent_dim = latent_dim),
        loss_weights
       )

## Step 5 - Train
### 5.1 - Using two nice custom callbacks :-)
Two custom callbacks are used:
 - `ImagesCallback` : qui va sauvegarder des images durant l'apprentissage - See [ImagesCallback.py](./modules/callbacks/ImagesCallback.py)
 - `BestModelCallback` : qui sauvegardera le meilleur model - See [BestModelCallback.py](./modules/callbacks/BestModelCallback.py)

In [ ]:
save_dir = "./run/models_dir/"
BestModelCallback   = BestModelCallback(dirpath = save_dir)
CallbackImages      = ImagesCallback(x=x_data, z_dim=latent_dim, nb_images=5, from_z=True, from_random=True, run_dir=run_dir)
logger              = TensorBoardLogger(save_dir='VAE2_logs',name="VAE_logs")

### 5.2 - Let's train !
With `scale=1`, need 1'15 on a GPU (V100 at IDRIS) ...or 20' on a CPU  

In [ ]:
chrono=fidle.Chrono()
chrono.start()

# train model
trainer= pl.Trainer(accelerator='auto',
                    max_epochs=epochs,
                    logger=logger,
                    num_sanity_val_steps=0,
                   callbacks=[CustomTrainProgressBar(), BestModelCallback, CallbackImages]
                   )

trainer.fit(model=vae, train_dataloaders=train_loader)



chrono.show()

## Step 6 - Training review
### 6.1 - History

In [ ]:
# launch Tensorboard 
%reload_ext tensorboard
%tensorboard --logdir=./VAE1_logs/VAE_logs/ --bind_all

### 6.2 - Reconstruction during training
At the end of each epoch, our callback saved some reconstructed images.  
Where :  
Original image -> encoder -> z -> decoder -> Reconstructed image

In [ ]:
images_z, images_r = CallbackImages.get_images( range(0,epochs,2) )

fidle.utils.subtitle('Original images :')
fidle.scrawler.images(x_data[:5], None, indices='all', columns=5, x_size=2,y_size=2, save_as='02-original')

fidle.utils.subtitle('Encoded/decoded images')
fidle.scrawler.images(images_z, None, indices='all', columns=5, x_size=2,y_size=2, save_as='03-reconstruct')

fidle.utils.subtitle('Original images :')
fidle.scrawler.images(x_data[:5], None, indices='all', columns=5, x_size=2,y_size=2, save_as=None)


### 6.3 - Generation (latent -> decoder) during training

In [ ]:
fidle.utils.subtitle('Generated images from latent space')
fidle.scrawler.images(images_r, None, indices='all', columns=5, x_size=2,y_size=2, save_as='04-encoded')

## Step 7 - Model evaluation

### 7.1 - Reload best model

In [ ]:
BestModelCallback.best_model_path

In [ ]:
#---- Load the model from a checkpoint
vae = VAE.load_from_checkpoint(BestModelCallback.best_model_path,
                                encoder=Encoder(latent_dim=latent_dim),
                                decoder=Decoder(latent_dim=latent_dim)
                              )
# put model in evaluation mode
vae.eval()


### 7.2 - Image reconstruction

In [ ]:
# ---- Select few images

x_show = fidle.utils.pick_dataset(x_data, n=10)

# ---- Get latent points and reconstructed images

z_mean, z_var, z  = vae.encoder(x_show.to(device))
x_reconst         = vae.decoder(z)

# ---- Show it
z         = z.cpu().detach()
x_reconst = x_reconst.cpu().detach()

labels=[ str(np.round(z[i],1)) for i in range(10) ]

fidle.scrawler.images(x_show,    None, indices='all', columns=10, x_size=2,y_size=2, save_as='05-original')
fidle.scrawler.images(x_reconst, None, indices='all', columns=10, x_size=2,y_size=2, save_as='06-reconstruct')


### 7.3 - Visualization of the latent space

In [ ]:
n_show = int(20000*scale)

# ---- Select images

x_show, y_show = fidle.utils.pick_dataset(x_data,y_data, n=n_show)

# ---- Get latent points

z_mean, z_var, z = vae.encoder(x_show.to(device))

# ---- Show them
z         = z.cpu().detach()           # Move the tensor to CPU and detach it
x_reconst = x_reconst.cpu().detach()

fig = plt.figure(figsize=(14, 10))
plt.scatter(z[:, 0] , z[:, 1], c=y_show, cmap= 'tab10', alpha=0.5, s=30)
plt.colorbar()
fidle.scrawler.save_fig('07-Latent-space')
plt.show()

### 7.4 - Generative latent space

In [ ]:
if latent_dim>2:

    print('Sorry, This part can only work if the latent space is of dimension 2')

else:
    
    grid_size   = 18
    grid_scale  = 1

    # ---- Draw a ppf grid

    grid=[]
    for y in scipy.stats.norm.ppf(np.linspace(0.99, 0.01, grid_size),scale=grid_scale):
        for x in scipy.stats.norm.ppf(np.linspace(0.01, 0.99, grid_size),scale=grid_scale):
            grid.append( (x,y) )
    grid=np.array(grid)

    # ---- Draw latentspoints and grid

    fig = plt.figure(figsize=(10, 8))
    plt.scatter(z[:, 0] , z[:, 1], c=y_show, cmap= 'tab10', alpha=0.5, s=20)
    plt.scatter(grid[:, 0] , grid[:, 1], c = 'black', s=60, linewidth=2, marker='+', alpha=1)
    fidle.scrawler.save_fig('08-Latent-grid')
    plt.show()

    # ---- Plot grid corresponding images
    grid=torch.from_numpy(grid).to(device)
    x_reconst = vae.decoder([grid])
    x_reconst = x_reconst.cpu().detach()
    fidle.scrawler.images(x_reconst, indices='all', columns=grid_size, x_size=0.5,y_size=0.5, y_padding=0,spines_alpha=0.1, save_as='09-Latent-morphing')



In [ ]:
fidle.end()

---
<img width="80px" src="../fidle/img/logo-paysage.svg"></img>